#**3장. 머신 러닝 핵심 알고리즘**

**3.1 지도 학습**

    지도 학습 알고리즘
    3.1.1 K-최근접 이웃
    3.1.2 서포트 벡터 머신
    3.1.3 결정 트리
    3.1.4 로지스틱 회귀와 선형 회귀

**3.2 비지도 학습**

    비지도 학습 알고리즘
    3.2.1 K-평균 군집화
    3.2.2 밀도 기반 군집 분석
    3.2.3 주성분 분석(PCA)

# 3.1 지도 학습

지도 학습 : 정답(레이블(label))을 컴퓨터에 미리 알려 주고 데이터를 학습시키는 방법

- 분류 : 주어진 데이터를 정해진 범주에 따라 분류
  1. 데이터 유형 : 이산형 데이터
  2. 결과 : 훈련 데이터의 레이블(정답) 중 하나 예측
  3. 예시 : 학습 데이터를 A,B,C 그룹 중 하나로 매핑 (ex: 스팸 메일 필터링)

- 회귀 : 데이터들의 특성을 기준으로 연속된 값을 그래프로 표현하여 패턴이나 트렌드를 예측할 떄 사용
  1. 데이터 유형 : 연속형 데이터
  2. 결과 : 연속된 값을 예측
  3. 예시 : 결괏값이 어떤 값으로든 나올 수 있음 (ex: 주가 분석 예측)

***지도 학습의 알고리즘***

**3.1.1 K-최근접 이웃 (K-nearest neighbor**)

: 새로운 입력을 받았을 때, 기존 데이터와의 거리를 계산하고 가장 가까운 데이터 K개를 선택하여 해당 클러스터로 할당하는 분류 알고리즘

- 과거 데이터를 저장해두고, 필요할 때마다 비교를 수행하는 방식 (미리 분류 모형 만들어둠 XX)
- K값에 따라 성능이 달라질 수 있으므로 초기 설정이 중요함
- 예) K=3일 때
  
  1)주변 범주 3개가 주황색 -> 주황색으로 분류

  2)주변 범주 두 개가 녹색, 하나가 주황색 -> 녹색으로 분류

In [14]:
#붓꽃에 대한 분류 - K-최근접 이웃 알고리즘을 이용하여
#목표 : 적당한 K값 예측

#1. 라이브러리 호출
import numpy as np #벡터 및 행렬의 연산 처리를 위한 라이브러리
import matplotlib.pyplot as plt #데이터를 차트나 플롯(plot)으로 그려주는 라이브러리
import pandas as pd #데이터 분석 및 조작을 위한 라이브러리
from sklearn import metrics #모델 성능 평가

In [15]:
#2. 데이터셋 준비
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'Class'] #열(column) 이름 할당

dataset = pd.read_csv('/content/drive/MyDrive/pytorch_ex/chap03/data/iris.data', names=names) #데이터를 판다스 데이터프레임에 저장
print(dataset.shape)

(150, 5)


In [18]:
#3. 훈련세트 데이터세트 분리
x = dataset.iloc[:, :-1].values #모든 행, 뒤에서 하나 뺀 열 선택
y = dataset.iloc[:, -1].values #모든 행, 마지막 열 선택

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

from sklearn.preprocessing import StandardScaler
s = StandardScaler() #특성 스케일링(scaling). 평균0 분산1 되도록 표준화 전처리
s.fit(X_train)
x_train = s.transform(x_train)
x_test = s.transform(x_test)

In [19]:
#4. 모델 생성 및 훈련
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=50) #K=50인 K-최근접 이웃 모델 생성
knn.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=50)

In [20]:
#5. 모델 정확도 측정
from sklearn.metrics import accuracy_score

y_pred = knn.predict(x_test)
print(f'정확도: { accuracy_score(y_test, y_pred)}')

정확도: 0.9333333333333333


In [24]:
#6. 최적의 K값 찾기
k = 10
acc_array = np.zeros(k)

for k in np.arange(1, k+1) : #k값을 1~10으로 반복
  classifier = KNeighborsClassifier(n_neighbors=k) #모델 생성
  classifier = classifier.fit(x_train, y_train) #모델 훈련
  y_pred = classifier.predict(x_test) #테스트세트 예측값
  acc = metrics.accuracy_score(y_test, y_pred) #정확도
  acc_array[k-1] = acc #측정한 정확도를 acc_array배열에 추가

max_acc = np.amax(acc_array) #가장 큰 정확도(최적의 정확도)
acc_list = list(acc_array)
k = acc_list.index(max_acc) + 1 #최적의 K값
print(f'정확도 {max_acc}으로 최적의 k는 {k}입니다')

정확도 1.0으로 최적의 k는 1입니다


**3.1.2 서포트 벡터 머신 (Support Vector Machine, SVM)**

: 분류를 위한 기준선을 정의하는 모델

    - 결정 경계 : 데이터를 분류하기 위한 기준선
    - 마진(margin) : 결정 경계와 서포트 벡터 사이의 거리
    - 서포트 벡터(support vector) : 결정 경계와 가까이 있는 데이터들
    - 이상치(outlier) : 패턴에서 벗어난 값
    - 하드 마진(hard margin) : 이상치를 허용하지 않는 것
    - 소프트 마진(soft margin) : 어느정도의 이상치가 마진 안에 포함되는 것을 허용하는 것

    - 커널 트릭(kernel trick) : 수학적 기교를 통해 저차원 데이터를 고차원으로 보내어 비선형 분류 문제를 해결하고자 도입
    - 선형 커널(linear kernel) : 선형으로 분류 가능한 데이터에 적용(기본 커널 트릭, 커널 트릭 사용Xx)
    - 다항식 커널(polynomial kernel) : 입력 벡터를 고차원으로 매핑하여 비선형 분류가 필요한 데이터에 적용(차수에 한계 Oo)
    - 가우시안 RBF 커널(Gaussian RBF kernel) : 입력 벡터를 무한한 고차원으로 매핑하여 비선형 분류가 필요한 데이터에 적용(차수 한계 Xx)


  - 결정 경계는 데이터가 분류된 클래스에서 가장 멀리 떨어져 있을 때(=마진 크기 최대화) 성능이 가장 좋음
  - 분류의 종류 : 선형 분류, 비선형 분류(저차원 데이터를 고차원으로 보내서 문제 해결)
  - 선형 커널과 가우시안 RBF 커널은 실제로 특성을 추가하는 게 아닌, 다양한 특성 조합이 생기는 것과 같은 효과를 구현하는 것

In [34]:
#붓꽃에 대한 분류 - 서포트 벡터 머신 알고리즘을 이용하여
#목표 : 네 가지 특성으로 붓꽃(iris) 분류

#1. 라이브러리 호출
from sklearn import svm #서포트 벡터 머신 라이브러리
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import datasets
from sklearn import model_selection
import tensorflow as tf
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [28]:
#2. 데이터셋 준비 및 분리
iris = datasets.load_iris() #사이킷런 제공 iris 데이터 호출
x_train, x_test, y_train, y_test = model_selection.train_test_split(iris.data, iris.target, test_size=0.6, random_state=42)

In [35]:
#3. 모델 생성, 훈련, 정확도

svm = SVC(kernel='linear', C=1.0, gamma=0.5)
#커널-선형 커널(선형 분류)  /  C-오류 어느정도 허용할지(클수록 하드마진, 작을수록 소프트마진)  /  감마-결정 경계를 얼마나 유연하게 할지(클수록 과적합 초래할 수도)

svm.fit(x_train, y_train)
y_pred = svm.predict(x_test)
score = metrics.accuracy_score(y_test, y_pred)

print(f'정확도: {score}')

정확도: 0.9888888888888889


**3.1.3 결정 트리(decision tree)**

: 데이터를 1차로 분류한 후, 각 영역의 순도(homogeneity)는 증가하고 불순도(impurity) 및 불확실성(uncertainity)은 감수하는 방향으로 학습을 진행

    - 정보 획득(information gain) : 순도가 증가하고 불확실성이 감소하는 것
    - 순도 : 범주 안에서 같은 종류의 데이터만 모여 있는 상태
    - 불순도 : 범주 안에 다른 종류의 데이터가 섞여 있는 상태
    - 랜덤 포레스트(random forest) : 결정 트리를 여러 개 묶어 놓은 것
  - 결정 트리에서 불확실성을 계산하는 방법
    1. 엔트로피(entropy) : 확률변수의 불확실성을 수치로 나타낸 것.
      - " 엔트로피 = 0 " == 불확실성 최소 == 순도 최대
      - " 엔트로피 = 0.5 " ==  불확실성 최대 == 순도 최소
    2. 지니 계수(Gini index) : 불순도를 측정하는 지표. 임의의 원소 2개가 서로 다른 그룹에 속해 있을 확률
      - 지니 계수가 높을수록 데이터가 분산되어 있음
      - 로그 계산이 없어 엔트로피보다 연산 빠름

In [46]:
#타이타닉 승객 생존 여부 예측 - 결정 트리 알고리즘을 이용하여

#1. 라이브러리 호출
import pandas as pd

#2. 데이터 준비
df = pd.read_csv('/content/drive/MyDrive/pytorch_ex/chap03/data/titanic/train.csv', index_col='PassengerId')
print(df.head())

             Survived  Pclass  ... Cabin Embarked
PassengerId                    ...               
1                   0       3  ...   NaN        S
2                   1       1  ...   C85        C
3                   1       3  ...   NaN        S
4                   1       1  ...  C123        S
5                   0       3  ...   NaN        S

[5 rows x 11 columns]


In [47]:
#3. 데이터 전처리
df = df[ ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']] #해당 열만 사용
df['Sex'] = df['Sex'].map( {'male':0, 'female':1} ) #성별 값을 0,1의 정수값으로 변환
df = df.dropna() #값이 없는 데이터 삭제
x = df.drop('Survived', axis=1) #삭제할 레이블 명 / 해당 레이블의 축(axis=1로 '열'임을 알려줌)
y = df['Survived']
print(x.head())
print(y.head())

             Pclass  Sex   Age  SibSp  Parch     Fare
PassengerId                                          
1                 3    0  22.0      1      0   7.2500
2                 1    1  38.0      1      0  71.2833
3                 3    1  26.0      0      0   7.9250
4                 1    1  35.0      1      0  53.1000
5                 3    0  35.0      0      0   8.0500
PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64


<ipython-input-47-a0c13c55219a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sex'] = df['Sex'].map( {'male':0, 'female':1} ) #성별 값을 0,1의 정수값으로 변환


In [48]:
#3. 데이터 전처리 - 훈련 세트와 테스트 세트 분리
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

In [49]:
#4. 결정 트리 모델 생성
from sklearn import tree
model = tree.DecisionTreeClassifier()

In [50]:
#5. 모델 훈련
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [52]:
#6. 모델 예측
y_pred = model.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8324022346368715

In [53]:
#7. 혼동 행렬을 이용한 성능 측정
from sklearn.metrics import confusion_matrix
conf_matrix = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns = ['Predicted Not Survival', 'Predicted Survival'],
    index = ['True Not Survival', 'True Survival']
)
print(conf_matrix)

                   Predicted Not Survival  Predicted Survival
True Not Survival                      99                  13
True Survival                          17                  50
